# Knapsack problem
This is a very common combinatorial optimisation problem where you are given a knapsack of a given weight capacity $C$ and a bunch of items with values and weight. The goal is to fill the knapsack with the best aggregated value, respecting the weight constraint.

![knapsack problem illustration](https://upload.wikimedia.org/wikipedia/commons/f/fd/Knapsack.svg "Image from wikipedia: https://commons.wikimedia.org/wiki/File:Knapsack.svg").

We handle here the *0-1 knapsack problem* where each item can only be taken once.


[explanation about choices of usecases and solvers needed]

## Prerequisites

Before running this notebook, you need to 
- install [minizinc](https://www.minizinc.org/) and config it so that it is found by the jupyter kernel (on linux, it means updating the `PATH` variable)
- install discrete-optimization in your jupyter kernel
    ```
    pip install discrete-optimization
    ```



### Imports

In [ ]:
import nest_asyncio
import logging
import time
from pprint import pprint

from discrete_optimization.datasets import fetch_data_from_coursera
from discrete_optimization.knapsack.knapsack_model import (
    KnapsackModel,
    KnapsackSolution,
)
from discrete_optimization.knapsack.solvers import (
    cp_solvers,
    lp_solvers,
    greedy_solvers,
)
from discrete_optimization.knapsack.knapsack_parser import (
    get_data_available,
    parse_file,
)
from discrete_optimization.knapsack.solvers.lp_solvers import (
    LPKnapsack,
    LPKnapsackCBC,
    MilpSolverName,
    ParametersMilp,
)
from discrete_optimization.generic_tools.cp_tools import ParametersCP, CPSolverName
from discrete_optimization.knapsack.solvers.cp_solvers import (
    CPKnapsackMZN2,
)
from discrete_optimization.generic_tools.do_problem import get_default_objective_setup
from discrete_optimization.knapsack.solvers.knapsack_lns_cp_solver import (
    ConstraintHandlerKnapsack,
)
from discrete_optimization.knapsack.solvers.knapsack_lns_solver import (
    InitialKnapsackMethod,
    InitialKnapsackSolution,
)
from discrete_optimization.generic_tools.lns_cp import LNS_CP

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [coursera](https://github.com/discreteoptimization/assignment).

In [ ]:
needed_datasets = ["ks_60_0", "ks_500_0"]
files_available_paths = get_data_available()

download_needed = False
for dataset in needed_datasets:
    if len([f for f in files_available_paths if dataset in f]) == 0:
        download_needed = True
        break

if download_needed:
    fetch_data_from_coursera()

## Small example: 60 items

To begin with, we use the dataset [ks_60_0](https://github.com/discreteoptimization/assignment/blob/master/knapsack/data/ks_60_0) from coursera.

### Parse input data

We parse the dataset file to load it as a discrete-optimization problem. In this case we get a `discrete_optimization.knapsack.knapsack_model.KnapsackModel`.

In [ ]:
files_available_paths = get_data_available()
model_file = [f for f in files_available_paths if "ks_60_0" in f][0]
model = parse_file(model_file, force_recompute_values=True)
print(type(model))

Here is a representation of the corresponding model.

In [ ]:
print(model)

We can get a first solution which respect the constraint (but of course is not optimal) by not taking any item.

In the following representation of a solution:
- "Value" is the aggregated values of the taken items, 
- "Weight" is the aggregated weight of the taken items, which should respect the knapsack capacity constraint
- "Taken" is a list of number of items taken for each type. For instance [0, 1, 0, ...] means that
  - item 0 is not taken
  - item 1 is taken
  - item 2 is not taken
  - ...

In [ ]:
solution = model.get_dummy_solution()
print(solution)

### Solve

#### Greedy solver

[we need a small description here. And why to choose this one.]

The first solver we try here is the greedy solver which is very fast but sub-optimal. The solution it will find is not necessarily the best possible solution, but it will respect the constraints.

The greedy method consists in sorting the items by density and trying to fill the knapsack starting by the denser items. We stop when further items cannot respect the capacity constraint.

In this version we also try to penalize heavy items by using a modified density penalized by the item weight, in order to avoid having the knapsack filled with very few items. The compute a solution for each density version and the one giving the best value is selected.

We first intialize the solver.

In [ ]:
greedy_solver = greedy_solvers.GreedyBest(knapsack_model=model)

We run it.

In [ ]:
results_greedy = greedy_solver.solve()

We retrieve and display the best solution found by the greedy solver.

In [ ]:
print(results_greedy.get_best_solution())

#### LP solver: Branch-and-Cut

[we need a small description here. And why to choose this one.]

We use here a solver which is a wrap around CBC solver of [mip python library](https://python-mip.readthedocs.io/en/latest/intro.html), itself a wrap around [COIN-OR Branch-and-Cut solver - CBC](https://github.com/coin-or/Cbc).
This is a solver of the family of Linear Programming (LP) solvers.
Unlike the previous one, this is an exact solver, which means it will find the actual optimum.


In [ ]:
lp_solver_cbc = LPKnapsack(knapsack_model=model, milp_solver_name=MilpSolverName.CBC)

In [ ]:
params_milp = ParametersMilp(
    time_limit=100,
    pool_solutions=10000,
    mip_gap_abs=0.0001,
    mip_gap=0.001,
    retrieve_all_solution=False,
    n_solutions_max=10000,
)
results_cbc = lp_solver_cbc.solve(parameters_milp=params_milp)

In [ ]:
print(results_cbc.get_best_solution())

#### LP solver: Gurobi  (optional)

If you have a license for [gurobi](https://www.gurobi.com/), you can also use it to solve the knapsack problem. 

In [ ]:
lp_solver_gurobi = LPKnapsack(knapsack_model=model, milp_solver_name=MilpSolverName.GRB)
params_milp = ParametersMilp(
    time_limit=100,
    pool_solutions=10000,
    mip_gap_abs=0.0001,
    mip_gap=0.001,
    retrieve_all_solution=False,
    n_solutions_max=10000,
)
results_gurobi = lp_solver_gurobi.solve(parameters_milp=params_milp)

In [ ]:
print(results_gurobi.get_best_solution())

## Bigger problem : 500 items

We try now to deal with a bigger example corresponding to ks_500_0 file. This is known to have optimal value "54939".

### Parse input data

In [ ]:
files_available_paths = get_data_available()
model_file = [f for f in files_available_paths if "ks_500_0" in f][0]
big_model = parse_file(model_file, force_recompute_values=True)

### Solve

#### LP solver: Branch-and-Cut

We try again the CBC solver.

[why not the greedy one?]

In [ ]:
lp_solver_cbc = LPKnapsack(
    knapsack_model=big_model, milp_solver_name=MilpSolverName.CBC
)
params_milp = ParametersMilp(
    time_limit=100,
    pool_solutions=10000,
    mip_gap_abs=0.0001,
    mip_gap=0.001,
    retrieve_all_solution=False,
    n_solutions_max=10000,
)
results_big_cbc = lp_solver_cbc.solve(parameters_milp=params_milp)

In [ ]:
print(results_big_cbc.get_best_solution())

#### LNS solver

[small description needed. Why only for bigger problem? explanation about params_cp choice]

In [ ]:
params_objective_function = get_default_objective_setup(problem=big_model)
params_cp = ParametersCP.default()
params_cp.TimeLimit = 10
params_cp.TimeLimit_iter0 = 1
solver = CPKnapsackMZN2(
    big_model,
    cp_solver_name=CPSolverName.GECODE,
    params_objective_function=params_objective_function,
)
solver.init_model()
initial_solution_provider = InitialKnapsackSolution(
    problem=big_model,
    initial_method=InitialKnapsackMethod.DUMMY,
    params_objective_function=params_objective_function,
)
constraint_handler = ConstraintHandlerKnapsack(problem=big_model, fraction_to_fix=0.8)
lns_solver = LNS_CP(
    problem=big_model,
    cp_solver=solver,
    initial_solution_provider=initial_solution_provider,
    constraint_handler=constraint_handler,
    params_objective_function=params_objective_function,
)
result_store = lns_solver.solve_lns(parameters_cp=params_cp, nb_iteration_lns=200)

## Conclusion